# BLADE DEMO


In this notebook, a demo of BLADE along with generation of simulation dataset is provided.

In addition to BLADE, the following two baseline methods are implemented:
- Non-negative least square (NNLS)
- Support Vector Regression (SVR)


### Load necessary modules


In [137]:
import sys, os
sys.path.append(os.path.relpath('../python/')) # location of python script

from BLADE import BLADE_framework as BLADE #to be changed
import numpy as np
from numpy import transpose as t
import itertools
import pickle
from scipy.optimize import nnls
from sklearn.svm import SVR
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error as mse
import pandas as pd

# modules for visualization
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import qgrid

## Generation of simulation data 

A simulation data is generated based on the varying setting of:
- number of cell types (`Ncells`)
- number of genes (`Ngenes`)
- number of samples (`Nsamples`)
- level of variability (`Noises`)

User can add or remove options by modifying the list variables below:

In [2]:
# training data
Ncells = [5]
Ngenes = [500]
Nsamples = [10, 20]
Noises = [0.5, 1, 1.5] 

A simulation dataset will be created for every possible combination of the above parameters.  
The script below will create `../simulationdata.pickle` that contains a dictionary object.

For each configuration of `Ncell`, `Ngene`, `Nsample` and `Noise`, the following elements are created:
- `F`: fraction of cell type (`Ncell` by `Nsample` matrix)
- `Mu`: average gene expression profile per cell type (signature in log-scale; `Ngene` by `Ncell` matrix)
- `Omega`: gene expression variability per cell type (`Ngene` by `Ncell` matrix)
- `X`: gene expression profile per cell type in each sample (`Nsample` by `Ngene` by `Ncell` multidimensional array)
- `Y`: bulk gene expression profile (log-scale; `Ngene` by `Nsample` matrix)



A rough illustration of the generation procedure follows:

1. Fraction of cell types (`F`) is generated from the Dirichlet distribution $F\sim\mathcal{D}(5)$)
2. Per cell type gene expression profiles (`Mu`) is generated from the normal distribution $Mu\sim\mathcal{N}(0, 2)$. The sampled gene expression profiles is in log-scale.
3. Given variation level (`Noise`), gene expression profile per cell type in each sample (`X`) is generated from the normal distribution $X\sim\mathcal{N}(\mu=Mu, \Omega=Noise)$
4. Bulk gene expression profile (`Y`) is calculated as follows: $Y = \log(F\cdot \exp(X)+1)$


In [3]:
simfile = '../simulationdata.pickle'
if not os.path.exists(simfile):
    Synthetic_data = dict()
else:
    Synthetic_data = pickle.load(open(simfile, 'rb'))

for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises
            ):
    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    
    if not name in list(Synthetic_data.keys()):
        F = np.random.dirichlet(np.ones(Ncell)*5, Nsample).transpose()
        Mu = np.random.normal(0, 2, size=(Ngene,Ncell))
        Omega = np.ones((Ngene,Ncell)) * Noise

        X = np.random.normal(Mu, Omega_train, size=(Nsample, Ngene, Ncell))
        Y = np.zeros((Ngene, Nsample))
        for i in range(Nsample):
            Y[:,i] = np.log(np.dot(np.exp(X[i,:,:]), Coef[:,i])+1)

    
        Synthetic_data[name] = {
            'Coef': F,
            'Mu' : Mu,
            'X' : X,
            'Y' : Y,
            'Omega' : Omega
        }
        
with open(simfile, 'wb') as fp:
    pickle.dump(Synthetic_data, fp, protocol=pickle.HIGHEST_PROTOCOL)

## Application of deconvolution methods

From here, we will apply the following three methods:
1. BLADE (estimation of cellular fraction + group-mode/high-resolution-mode purification)
2. NNLS (estimatino of fraction)
3. SVR followed by NNLS (estimation of fraction + group-mode purification) - similar to CIBERSORTx


### 1. Application of BLADE

There are following key paramters used in BLADE:
- Hyperparameters (`hyperpars`): `Alpha`, `Alpha0`, `Kappa0` and `SigmaY`, each of which can be defined as a list of options. BLADE takes an empirical Bayes approach to find the optimal parameter set given the all possible combinations. 
- `Nrep`: Number of repeat for evaluating each parameter configuration.
- `Nrepfinal`: Number of repeated optimization for the final parameter set.
- `Njob`: Number of parallel jobs.

In [4]:
hyperpars = {
    'Alpha': [1, 10],
    'Alpha0': [0.1, 1, 5],
    'Kappa0': [1, 0.5, 0.1],
    'SigmaY': [0.3, 0.5, 1]
}

Nrep=3
Nrepfinal=20
Njob=50

Given the configuration above, BLADE is applied to each of the simulation dataset created above.  

BLADE produce several outcomes:
- `final_obj`: final BLADE object with optimized variational parameters
- `best_obj`: BLADE object traind with the best parameter set found by the empirical Bayes framework. Empirical Bayes framework is applied after selecting a subset of samples (5 samples; indicated by `Ind_sample` below), and thus the outcome contains only 5 samples. If `Nsample` <= 5, `final_obj` is identical with `best_obj`.
- `best_set`: Best parameter set defined by Empirical Bayes framework.
- `outs`: Outcome of BLADE for every possible combination of hyperparameters, used in the Empirical Bayes framework. 


In [5]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/BLADE_outcome_' + name + '.pickle'

    if not os.path.exists(outfile):
        print('creating ' + outfile)
        Y = np.exp(Synthetic_data[name]['Y']) - 1 #transform to the original scale
        mean = Synthetic_data[name]['Mu']
        sd = Synthetic_data[name]['Omega']
        
        # samples that are used in Empirical Bayes step
        Ind_sample = [True]*5 + [False]*(Nsample - 5) 
        # indicator variables for the genes used in fraction estimations. 
        # In purification step, all genes are used
        Marker_Index = [True] * Ngene
        
        final_obj, best_obj, best_set, outs = BLADE(
            mean, sd, Y, Marker_Index, Ind_sample,
            hyperpars['Alpha'], hyperpars['Alpha0'], hyperpars['Kappa0'], hyperpars['SigmaY'],
            Nrep=Nrep, Njob=Njob, Nrepfinal=Nrepfinal, fsel=0)
        
        pickle.dump(
            {
                'final_obj': final_obj,
                'best_obj': best_obj,
                'best_set': best_set,
                'outs' : outs
            },
                open(outfile, 'wb')
        )

### 2. Application of NNLS 

Non-negative least square is applied to obtain cellular fraction (`F`) using bulk gene expression profiles (`Y`) and gene expression sigantures (`Mu`; average gene expression profiles per cell type). Note that NNLS should be applied to linear-scale data. The simulation data in `Synthetic_data` are in log-scale ($f(x)=\log(x+1)$ for `Y` and $f(x)=\log(x)$ for `Mu`), so they are transformed back to the linear scale.

In [6]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/NNLS_outcome_' + name + '.pickle'

    if not os.path.exists(outfile):
        
        print('creating ' + outfile)
        Y = np.exp(Synthetic_data[name]['Y'])-1 # transform back to linear scale
        Mu = np.exp(Synthetic_data[name]['Mu']) # transform back to linear scale
        
        NNLS_F = np.zeros(Synthetic_data[name]['Coef'].shape)
        for i in range(Nsample):
            NNLS_F[:,i] = nnls(Mu, Y[:,i])[0]
            NNLS_F[:,i] = NNLS_F[:,i]/np.sum(NNLS_F[:,i])
        
        pickle.dump(
            {
                'Fraction': NNLS_F
            },
            open(outfile, 'wb')
        )

### 3. Application of SVR followed by NNLS (SVR_NNLS)

SVR has been applied for estimation of cellular fractions (`Fraction`). As in CIBERSORT and CIBERSORTx, we used here Nu-SVR where the `Nu` controls the number of genes used in deconvolution. In CIBERSORTx, a sequencial approach is taken to also estimate gene expression profiles per cell type (both group-mode purification and high-resolution-mode purification). Here, we implement only the group-mode-purification (which we saved as `Signature`).

As in NNLS, both bulk gene expression profiles (Y) and gene expression sigantures (Mu; average gene expression profiles per cell type) should be transformed to linear-scale.


In [7]:
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    outfile = '../data/SVR_NNLS_outcome_' + name + '.pickle'

    if not os.path.exists(outfile):
        
        print('creating ' + outfile)
        Y = np.exp(Synthetic_data[name]['Y']) - 1
        Mu = np.exp(Synthetic_data[name]['Mu'])
   
        # estimate fraction
        SVRcoef = np.zeros((Ncell, Nsample))
        Selcoef = np.zeros((Ngene, Nsample))
        Nus = [0.25, 0.5, 0.75]
        for i in range(Nsample):
            sols = [NuSVR(kernel='linear', nu=nu).fit(Mu,Y[:,i]) for nu in Nus]
            RMSE = [mse(sol.predict(Mu), Y[:,i]) for sol in sols]
            Selcoef[sols[np.argmin(RMSE)].support_, i] = 1
            SVRcoef[:,i] = np.maximum(sols[np.argmin(RMSE)].coef_,0)
            SVRcoef[:,i] = SVRcoef[:,i]/np.sum(SVRcoef[:,i])
            
           
        # estimate per-cell expression
        NNLS_Sig = np.zeros((Ngene, Ncell))
        for g in range(Ngene):
            NNLS_Sig[g,:] = nnls(np.transpose(SVRcoef), Y[g,:])[0]
        
        pickle.dump(
            {
                'Fraction' : SVRcoef,
                'Signature': NNLS_Sig
             },
            open(outfile, 'wb')
         )

## Performance evaluation

We evaluated the performance for each data set in both per-cell and per-sample manner. In the function below (`measure_performance`), we implemented a function to collect the performances.

Specifically,
- Per cell type (`row_PerCell`):
 - `Cor_Fraction`: Pearson correlation coefficients of true (`true_f`) and estimated cellular fractions (`est_f`) per cell type across the samples.
 - `Cor_Signature`: Pearson correlation coefficients of true (`true_mean`) and estimated average gene expression profiles per cell type (Signatures; `est_mean`)
 - `Cor_HighRes`: Perason correaltion coefficients of true (`true_X`) and estimated gene expresion profile per cell type per sample (`est_X`). An average Pearson correlation per cell type is obtained.

- Per sample (`row_PerSample`):
 - `Cor_Fraction`: Pearson correlation coefficients of true (`true_f`) and estimated cellular fractions (`est_f`) per sample across the cell types.
 - `Cor_HighRes`: Perason correaltion coefficients of true (`true_X`) and estimated gene expresion profile per cell type per sample (`est_X`). An average Pearson correlation per sample is obtained.
 
- Summary (`row_Summary`):
 - Average performances of each measure in `row_PerCell` and `row_PerSample` are obtained. `Cor_HighRes` is not included twice as the average across the sample and cell types yield equal values.

In [19]:
def measure_performance(true_f, true_mean, true_X, 
                        est_f, est_mean=None, est_X=None):
    Nsample, Ngene, Ncell = true_X.shape
    
    # group-mode purification performance
    if est_mean is not None:
        CorMean = [np.corrcoef(true_mean[:,i], est_mean[:,i], rowvar=True)[0,1] for i in range(Ncell)]
    else:
        CorMean = np.nan*np.zeros((Ncell))
    
    # high-resolution purification performance
    if est_X is not None:
        perf_highresCell = [
                np.mean(
                    [np.corrcoef(true_X[sample,:,cell], est_X[sample,:,cell])[0,1] for sample in range(Nsample)]
                ) for cell in range(Ncell)
            ]
        perf_highresSample = [
                np.mean(
                    [np.corrcoef(true_X[sample,:,cell], est_X[sample,:,cell])[0,1] for cell in range(Ncell)]
                ) for sample in range(Nsample)
            ]
    else:
        perf_highresCell = [np.nan]*Ncell
        perf_highresSample = [np.nan]*Nsample
    
    # Performance: per cell type
    CorsF = [np.corrcoef(true_f[i,:], est_f[i,:], rowvar=True)[0,1] for i in range(Ncell)]
    row_PerCell = pd.DataFrame(np.array((range(Ncell), CorsF, CorMean, perf_highresCell)).transpose(),
                    columns = ['CellType', 'Cor_Fraction', 'Cor_Signature', 'Cor_HighRes'])
    
    # Performance: per sample
    CorsF = [np.corrcoef(true_f[:,i], est_f[:,i], rowvar=True)[0,1] for i in range(Nsample)]
    row_PerSample = pd.DataFrame(np.array((range(Nsample), CorsF, perf_highresSample)).transpose(),
                    columns = ['Sample', 'Cor_Fraction', 'Cor_HighRes'])
    
    # Performance - summary
    row_Summary = pd.DataFrame([[np.mean(row_PerCell.Cor_Fraction), 
                                        np.mean(row_PerSample.Cor_Fraction), 
                                        np.mean(row_PerCell.Cor_Signature),
                                        np.mean(row_PerSample.Cor_HighRes)
                               ]],
            columns = ['Cor_Fraction(PerCell)', 'Cor_Fraction(PerSample)', 
                     'Cor_Signature', 'Cor_HighRes']
            )
    return (row_PerCell, row_PerSample, row_Summary)

Using the function above, we will collect the performance metrics of the three methods.
For the evaluation, we need to obtain estimates of cellular fractions, average gene expression profile per cell type (Signature; only for BLADE and SVR_NNLS), and gene expression profile per cell type in each sample (only for BLADE).

Few specifics regarding the extraction of the estimates:
- BLADE
 - Fraction estimation: For each sample $i$ and cell type $t$, the fraction can be calcualted from the optimized variational parameter $\beta_i^t$ as follows: $\frac{\beta_i^t}{\sum_k(\beta_i^k)}$. From the final object (`final_obj`), it can be calculated by the internal function `ExpF`.
 - Group-mode purification: Variational parameter $\nu_{ij}^t$ represents expression level of gene $j$ of cell type $t$ in sample $i$. Thus, average gene expression per cell type is: $\frac{1}{Nsample}\sum_i \nu_{ij}^t$.
 - High-resolution mode purification: we can use $\nu_{ij}^t$ as the estimate.

- NNLS
 - Fraction estimation: Regression coefficients stands for the fraction.
 - Group-mode purification: Not available
 - High-resolution-mode purification: Not available

- SVR_NNLS
 - Fraction estimation: Regression coefficients from SVR.
 - Group-mode purification: Regression coefficients from NNLS.
 - High-resolution-mode purification: Not available

In [20]:
outcome_PerCell = pd.DataFrame()
outcome_PerSample = pd.DataFrame()
outcome_summary = pd.DataFrame()
                               
for Ncell, Ngene, Nsample, Noise in itertools.product(
    Ncells, Ngenes, Nsamples, Noises,
                ):

    name = str(Ncell) +'_'+ str(Ngene) +'_'+ str(Nsample) +'_'+ str(Noise)
    
    #Loading test data
    fraction = np.array(Synthetic_data[name]['Coef'])
    mean = np.array(Synthetic_data[name]['Mu'])
    X = np.array(Synthetic_data[name]['X'])
    
    # Load outcome
    BLADE = pickle.load(open('../data/BLADE_outcome_' + name + '.pickle', 'rb'))
    NNLS = pickle.load(open('../data/NNLS_outcome_' + name + '.pickle', 'rb'))
    SVR_NNLS = pickle.load(open('../data/SVR_NNLS_outcome_' + name + '.pickle', 'rb'))
    
    # Obtaining estimates from the three methods
    obj = BLADE['final_obj']
    
    outcomes = {
        'BLADE': {
            'Fraction': t(obj.ExpF(obj.Beta)), 
            'Signature': np.mean(obj.Nu, 0), #group mode purification
            'HighRes': obj.Nu                #highresolution mode purification
        },
        'NNLS' : {
            'Fraction': NNLS['Fraction'],
            'Signature': None, 'HighRes': None
        },
        'SVR_NNLS' : {
            'Fraction': SVR_NNLS['Fraction'],
            'Signature': np.log(SVR_NNLS['Signature']+0.0001),
            'HighRes': None
        }
    }
    
    # measure performance
    for method in outcomes:
        rowCell, rowSample, rowSummary = \
            measure_performance(
                fraction, mean, X,
                outcomes[method]['Fraction'], outcomes[method]['Signature'], outcomes[method]['HighRes']
            )
        rowCell['method'], rowSample['method'], rowSummary['method'] = [method]*3
        rowCell['Ncell'], rowSample['Ncell'], rowSummary['Ncell'] = [Ncell]*3
        rowCell['Ngene'], rowSample['Ngene'], rowSummary['Ngene'] = [Ngene]*3
        rowCell['Nsample'], rowSample['Nsample'], rowSummary['Nsample'] = [Nsample]*3
        rowCell['Noise'], rowSample['Noise'], rowSummary['Noise'] = [Noise]*3
        rowCell['dataset'], rowSample['dataset'], rowSummary['dataset'] = [name]*3
    
        # collect outcome
        outcome_PerCell = outcome_PerCell.append(rowCell)
        outcome_PerSample = outcome_PerSample.append(rowSample)
        outcome_summary = outcome_summary.append(rowSummary)

## Visualization of outcome


### 1. Performance of cellular fraction estimation

We will first visualize the performance measure per cell type.
First of all, a boxplot can be created for each data set as below.

In [181]:
fig = px.box(outcome_PerCell, y='Cor_Fraction', color='method', x='dataset', points="all",
            labels=dict(Cor_Fraction = "Pearson Correlation",
                        dataset = "Data Set (Ncell_Ngene_Nsample_Noise)"
                       )
            )

fig.update_layout(height=500, width=800,
                  title_text="Performance of fraction estimation (per cell type)")
fig.show()

Then, performance of each cell type can also be compared across the methods using a scatter plot.

In [178]:
PerfMat = outcome_PerCell.pivot_table(values='Cor_Fraction', columns='method', index=['CellType','dataset'])
PerfMat.reset_index(inplace=True)
PerfMat.dataset = PerfMat.dataset.astype('category')

colorsIdx = dict()
for name in set(PerfMat.dataset):
    # random RGB color generation for each dataset, then convert to hex
    colorsIdx[name] = '#%02x%02x%02x' % tuple(np.random.choice(range(256), size=3))
cols = PerfMat.dataset.map(colorsIdx)

fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Scatter(y=PerfMat.BLADE, x=PerfMat.NNLS, mode="markers", text=PerfMat.dataset,
               marker=dict(color=cols, coloraxis="coloraxis")
            ),
    row=1,col=1
)

fig.add_trace(
    go.Scatter(y=PerfMat.BLADE, x=PerfMat.SVR_NNLS, mode="markers", text=PerfMat.dataset,
               marker=dict(color=cols, coloraxis="coloraxis")
            ),
    row=1,col=2
)

fig.update_layout(height=500, width=1000,
                  title_text="Comparison of performance per cell type")
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)

fig.update_yaxes(range=[0, 1], title_text="BLADE")
fig.update_xaxes(range=[0, 1], title_text="NNLS", row=1,col=1)
fig.update_xaxes(range=[0, 1], title_text="SVR_NNLS", row=1,col=2)
fig.show()

Performance of fraction estimation can be also measured per sample.

In [180]:
fig = px.box(outcome_PerSample, y='Cor_Fraction', color='method', x='dataset', points="all",
            labels=dict(Cor_Fraction = "Pearson Correlation (fraction estimation performance)",
                        dataset = "Data Set (Ncell_Ngene_Nsample_Noise)"
                       )
            )
fig.update_layout(height=500, width=800,
                  title_text="Performance of fraction estimation (per sample)")
fig.show()

In [155]:
PerfMat = outcome_PerSample.pivot_table(values='Cor_Fraction', columns='method', index=['Sample','dataset'])
PerfMat.reset_index(inplace=True)
PerfMat.dataset = PerfMat.dataset.astype('category')
cols = PerfMat.dataset.map(colorsIdx)

fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Scatter(y=PerfMat.BLADE, x=PerfMat.NNLS, mode="markers", text=PerfMat.dataset,
               marker=dict(color=cols, coloraxis="coloraxis")
            ),
    row=1,col=1
)

fig.add_trace(
    go.Scatter(y=PerfMat.BLADE, x=PerfMat.SVR_NNLS, mode="markers", text=PerfMat.dataset,
               marker=dict(color=cols, coloraxis="coloraxis")
            ),
    row=1,col=2
)

fig.update_layout(height=500, width=1000,
                  title_text="Comparison of performance per sample")
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)

fig.update_yaxes(range=[0, 1], title_text="BLADE")
fig.update_xaxes(range=[0, 1], title_text="NNLS", row=1,col=1)
fig.update_xaxes(range=[0, 1], title_text="SVR_NNLS", row=1,col=2)
fig.show()

### 2. Performance of gene expression purification

Performance for estimation of gene expression profiles per cell type can be compared.  
Group-mode purification is done by BLADE and SVR_NNLS.  

In [182]:
fig = px.box(outcome_PerCell, y='Cor_Signature', color='method', x='dataset', points="all",
            labels=dict(Cor_Signature = "Pearson Correlation",
                        dataset = "Data Set (Ncell_Ngene_Nsample_Noise)"
                       )
            )

fig.update_layout(height=500, width=800,
                  title_text="Performance of group-mode purification (per cell type)")
fig.show()

In [183]:
PerfMat = outcome_PerCell.pivot_table(values='Cor_Signature', columns='method', index=['CellType','dataset'])
PerfMat.reset_index(inplace=True)
PerfMat.dataset = PerfMat.dataset.astype('category')

cols = PerfMat.dataset.map(colorsIdx)

fig = make_subplots(rows=1, cols=1)
fig.add_trace(
    go.Scatter(y=PerfMat.BLADE, x=PerfMat.SVR_NNLS, mode="markers", text=PerfMat.dataset,
               marker=dict(color=cols, coloraxis="coloraxis")
            ),
    row=1,col=1
)

fig.update_layout(height=500, width=500,
                  title_text="Comparison of performance per cell type")
fig.update_layout(coloraxis=dict(colorscale='Bluered_r'), showlegend=False)

fig.update_yaxes(range=[0, 1], title_text="BLADE")
fig.update_xaxes(range=[0, 1], title_text="SVR_NNLS", row=1,col=1)
fig.show()

Now, we will assess performance of BLADE for high-resolution mode purification.  
The performance can be measure per cell type (average across the samples) and per sample (average across the cell types)

In [176]:
cols = outcome_PerCell.dataset.map(colorsIdx)

fig = make_subplots(rows=1, cols=2, shared_yaxes=True)
fig.add_trace(
    go.Box(x=outcome_PerCell.dataset, y=outcome_PerCell.Cor_HighRes, 
           boxpoints="all", name="Average performance per cell type"),
    1,1
)

fig.add_trace(
    go.Box(x=outcome_PerSample.dataset, y=outcome_PerSample.Cor_HighRes, 
           boxpoints="all", name="Average performance per sample"),
    1,2
)

fig.update_layout(height=600, width=800,
                  title_text="High-resolution-mode purification")
fig.update_yaxes(range=[0, 1], title_text="Pearson Correlation")



Average performance per dataset can also be checked in the table below:


In [184]:
qgrid_widget = qgrid.show_grid(outcome_summary, show_toolbar=True)
qgrid_widget

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…